In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [2]:
#load data
block = pd.read_table("inversion_block_info_all.txt", sep='\t', header=None)
ATCC = pd.read_csv("Ecoli_ATCC_25922_final_exp.csv")
BW25113 = pd.read_csv("Ecoli_BW25113_final_exp0707.csv")
K12DH = pd.read_csv("Ecoli_K12_DH10B_final_exp.csv")
K12MG = pd.read_csv("Ecoli_K12_MG1655_final_exp.csv")

In [3]:
block.columns = ['blockname','taxa','start','end','rev_comp','inversion']

In [4]:
block['block'] = block['blockname'].apply(lambda x:x.split('.')[0])

In [7]:
block = block[['block', 'taxa','start','end','rev_comp','inversion']]

In [18]:
block['taxa'].value_counts()

CP009072       1148
NZ_CP009273    1148
NC_010473      1148
U00096000      1148
Name: taxa, dtype: int64

In [19]:
#sort block files based on taxa
ATCC_block = block[block.taxa=='CP009072'].sort_values(by='start').reset_index(drop=True)
BW25113_block = block[block.taxa=='NZ_CP009273'].sort_values(by='start').reset_index(drop=True)
K12DH_block = block[block.taxa=='NC_010473'].sort_values(by='start').reset_index(drop=True)
K12MG_block = block[block.taxa=='U00096000'].sort_values(by='start').reset_index(drop=True)

# #fixing rows that were not parse properly
# fix = block.taxa.unique()[4:]
# df = pd.DataFrame({})
# for i in fix:
#     df = df.append(block.loc[block.taxa==i])

# df['block2'] = df['block'].apply(lambda x : x.split('NZ')[1])
# df['block'] = df['block'].apply(lambda x : x.split('NZ')[0])

# df = df.drop(['inversion'],1)
# df.columns = ['block','start', 'end', 'rev_comp', 'inversion', 'taxa']
# df = df[['block', 'taxa', 'start', 'end','rev_comp', 'inversion']]

# df['taxa'] = df['taxa'].apply(lambda x : 'NZ'+x)

# #add fixed rows into the block files
# ATCC_block = ATCC_block.append(df.loc[df['taxa']=="NZ_CP009072"])
# BW25113_block = BW25113_block.append(df.loc[df['taxa']=="NZ_CP009273"])

ATCC_block['start'] = pd.to_numeric(ATCC_block['start'])
ATCC_block = ATCC_block.sort_values(by='start').reset_index(drop=True)

BW25113_block['start'] = pd.to_numeric(BW25113_block['start'])
BW25113_block = BW25113_block.sort_values(by='start').reset_index(drop=True)

In [20]:
#function to match by start
def match_block_start(gene_start, block_file): 
    block_file = block_file.sort_values(by='start').reset_index(drop=True)
    starts =[]
    for i in block_file.start:
        if i < gene_start: 
            starts.append(i)
    if len(starts) == 0:
        return None
    else:
        return(block_file.block[np.argmax(starts)])

#function to match by gene end 
def match_block_end(gene_end, block_file):
    block_file = block_file.sort_values(by='start', ascending=False).reset_index(drop=True)
    ends =[]
    for j in block_file.end:
        if j > gene_end: 
            ends.append(j)
    if len(ends)==0:
        return(block_file.block[len(block_file)-1])
    else:
        return(block_file.block[np.argmin(ends)])

## files to check neighbouring blocks

In [21]:
ATCC_block['block_order'] = np.arange(1,len(ATCC_block)+1,1)
BW25113_block['block_order'] = np.arange(1,len(BW25113_block)+1,1)
K12DH_block['block_order'] = np.arange(1,len(K12DH_block)+1,1)
K12MG_block['block_order'] = np.arange(1,len(K12MG_block)+1,1)

In [22]:
#number of rows check
len(ATCC_block)==len(BW25113_block)==len(K12DH_block)==len(K12MG_block)

True

# ATCC

### block file - spaces between blocks

In [23]:
##find difference between blocks
diff = [0]
for i in range(len(ATCC_block.start)-1):
    diff.append(ATCC_block.start[i+1] - ATCC_block.end[i])

ATCC_block['diff'] = diff
ATCC_block['overlaps'] = ATCC_block['diff'] < 0 

In [24]:
# block = ['a','b','c','d']
# start = [1,6,10,21]
# end = [20,9,12,25]

block = ATCC_block['block']
start = ATCC_block['start']
end = ATCC_block['end']

N = len(block)

tally =[]
for i in range(len(block)):
    tally.append((start[i], block[i],"start"))
    tally.append((end[i], block[i],"end"))
    
tally = sorted(tally, key=lambda x:x[0])

groups={}
stack={}
for entry in tally:
    t = entry[0]
    name = entry[1]
    action = entry[2]
    
    if action == "start":
        stack[name]= True
        if len(stack)>1:
            groups[','.join(stack)] = True
    if action == "end":
        del stack[name]

#print(tally)
#print(groups)
##Overlapping of 3 blocks, no inversion conflict
# print([i for i in groups.keys()][40])
# ATCC_block.loc[477:480]

In [25]:
ATCC_overlaps = pd.DataFrame({'Overlapping_blocks':[x for x in groups.keys()]})
blocks = ATCC_overlaps['Overlapping_blocks'].str.split(',',n=2, expand=True)
ATCC_overlaps['Bigblock'] = blocks[0]
ATCC_overlaps['Smallblock1'] = blocks[1]
ATCC_overlaps['Smallblock2'] = blocks[2]
ATCC_overlaps = pd.merge(ATCC_overlaps, ATCC_block[['block','start','end','inversion']], left_on='Bigblock',right_on='block', how='left')
ATCC_overlaps = pd.merge(ATCC_overlaps, ATCC_block[['block','start','end','inversion']], left_on='Smallblock1',right_on='block', how='left')
ATCC_overlaps['inversion_conflict'] = ATCC_overlaps['inversion_x'] != ATCC_overlaps['inversion_y']
ATCC_overlaps['inside_block'] = ATCC_overlaps['end_y'] < ATCC_overlaps['end_x']
print('there are', sum(ATCC_overlaps['inversion_conflict']), 'out of', len(ATCC_overlaps),'overlaps that have inconsistent inversion')
print(sum(ATCC_overlaps[ATCC_overlaps['inversion_conflict']]['inside_block']),'inconsitencies are inside blocks')
ATCC_overlaps.head()

there are 48 out of 68 overlaps that have inconsistent inversion
46 inconsitencies are inside blocks


,Overlapping_blocks,Bigblock,Smallblock1,Smallblock2,block_x,start_x,end_x,inversion_x,block_y,start_y,end_y,inversion_y,inversion_conflict,inside_block
0,"Block627,Block625",Block627,Block625,None,Block627,74379,98581,0,Block625,76379,76482,0,False,True
1,"Block627,Block626",Block627,Block626,None,Block627,74379,98581,0,Block626,76853,77129,0,False,True
2,"Block637,Block609",Block637,Block609,None,Block637,133236,140045,0,Block609,133257,133286,1,True,True
3,"Block637,Block515",Block637,Block515,None,Block637,133236,140045,0,Block515,139971,140002,1,True,True
4,"Block644,Block1080",Block644,Block1080,None,Block644,168457,172345,0,Block1080,168513,168548,1,True,True


## Overlapping gene check

In [26]:
overlapping_region = ATCC_overlaps[ATCC_overlaps['inside_block']==False].reset_index(drop=True)

overlapping_region['start'] = overlapping_region['start_y']
overlapping_region['end'] = overlapping_region['end_x']
overlapping_region['block'] = overlapping_region['Overlapping_blocks']
overlapping_region = overlapping_region[['block', 'start', 'end', 'inversion_conflict']]
overlapping_region 

,block,start,end,inversion_conflict
0,"Block709,Block710",463359,463448,False
1,"Block721,Block723",518405,518420,False
2,"Block824,Block823",1550682,1550866,False
3,"Block1036,Block1037",2146364,2146397,False
4,"Block79,Block1127",2511196,2511354,False
5,"Block27,Block28",2811080,2811133,False
6,"Block308,Block741",3876291,3876494,True
7,"Block741,Block307",3876516,3876582,True


In [27]:
#for overlap mapping
mapdf = ATCC_overlaps[ATCC_overlaps['inside_block']==False].reset_index(drop=True)
mapdf.head()

,Overlapping_blocks,Bigblock,Smallblock1,Smallblock2,block_x,start_x,end_x,inversion_x,block_y,start_y,end_y,inversion_y,inversion_conflict,inside_block
0,"Block709,Block710",Block709,Block710,None,Block709,452281,463448,0,Block710,463359,464582,0,False,False
1,"Block721,Block723",Block721,Block723,None,Block721,495769,518420,0,Block723,518405,529887,0,False,False
2,"Block824,Block823",Block824,Block823,None,Block824,1544980,1550866,1,Block823,1550682,1553430,1,False,False
3,"Block1036,Block1037",Block1036,Block1037,None,Block1036,2146159,2146397,0,Block1037,2146364,2150588,0,False,False
4,"Block79,Block1127",Block79,Block1127,None,Block79,2510755,2511354,0,Block1127,2511196,2521120,0,False,False


In [28]:
# blocs = mapdf[['Bigblock']]

# blocs2 = mapdf[['Smallblock1']]
# blocs2 = blocs2.rename(columns={'Smallblock1': 'Bigblock'})

# blocs = pd.concat([blocs,blocs2])

# len(blocs)

In [29]:
def overlapping_gene(start, end, block_file):
    inside_gene, overlapping_start, overlapping_end=[],[],[]
    for i in range(len(block_file)):
        if (block_file['gbk_start'][i] < start and block_file['gbk_end'][i] > start):
            overlapping_start.append(block_file['gene_id'][i])
        if (block_file['gbk_start'][i] > start and block_file['gbk_end'][i] < end):
            inside_gene.append(block_file['gene_id'][i]) 
        if (block_file['gbk_start'][i] > start and block_file['gbk_start'][i] < end):
            overlapping_end.append(block_file['gene_id'][i]) 
    return [overlapping_start,inside_gene,overlapping_end]

In [30]:
ATCC.gbk_end = [i.replace('>','') for i in ATCC.gbk_end]
ATCC['gbk_end'] = pd.to_numeric(ATCC['gbk_end'])
ATCC_regions = {}
regions={}
for i in range(len(overlapping_region)):
    regions[overlapping_region['block'][i]] = overlapping_gene(overlapping_region['start'][i], 
                                                               overlapping_region['end'][i], ATCC)
regions

{'Block709,Block710': [[], [], []],
 'Block721,Block723': [[], [], ['DR76_RS02385']],
 'Block824,Block823': [[], [], []],
 'Block1036,Block1037': [[], [], []],
 'Block79,Block1127': [[], [], []],
 'Block27,Block28': [['DR76_RS13700'], [], []],
 'Block308,Block741': [['DR76_RS18985'], [], []],
 'Block741,Block307': [['DR76_RS18985'], [], []]}

In [31]:
ATCC_overlaps.head()

,Overlapping_blocks,Bigblock,Smallblock1,Smallblock2,block_x,start_x,end_x,inversion_x,block_y,start_y,end_y,inversion_y,inversion_conflict,inside_block
0,"Block627,Block625",Block627,Block625,None,Block627,74379,98581,0,Block625,76379,76482,0,False,True
1,"Block627,Block626",Block627,Block626,None,Block627,74379,98581,0,Block626,76853,77129,0,False,True
2,"Block637,Block609",Block637,Block609,None,Block637,133236,140045,0,Block609,133257,133286,1,True,True
3,"Block637,Block515",Block637,Block515,None,Block637,133236,140045,0,Block515,139971,140002,1,True,True
4,"Block644,Block1080",Block644,Block1080,None,Block644,168457,172345,0,Block1080,168513,168548,1,True,True


## inside block genes check

### no inversion conflict

In [32]:
insideblock_region = ATCC_overlaps[ATCC_overlaps['inside_block']==True].reset_index(drop=True)
insideblock_region = insideblock_region[insideblock_region['inversion_conflict']==False].reset_index(drop=True)

insideblock_region['start'] = insideblock_region['start_y']
insideblock_region['end'] = insideblock_region['end_y']
insideblock_region['block'] = insideblock_region['Overlapping_blocks']
insideblock_region = insideblock_region[['block', 'start', 'end', 'inversion_conflict']]

In [33]:
inside_regions={}
for i in range(len(insideblock_region)):
    inside_regions[insideblock_region['block'][i]] = overlapping_gene(insideblock_region['start'][i], 
                                                                      insideblock_region['end'][i], ATCC)
pd.DataFrame(inside_regions)

,"Block627,Block625","Block627,Block626","Block678,Block65","Block1011,Block719","Block913,Block96","Block838,Block912","Block1044,Block56","Block1074,Block21","Block1074,Block21,Block1072","Block1123,Block78","Block1127,Block267","Block485,Block261","Block585,Block584","Block598,Block599"
0,[],[DR76_RS00355],[],[DR76_RS03020],[],[],[DR76_RS10610],[],[],[],[],[],[],[]
1,[],[],[],[],[],[],[],[],[],[],[],[],[],[]
2,[DR76_RS00355],[],[],[],[],[],[],[],[],[DR76_RS12165],[],[],[],[]


In [34]:
insideblock_region.head()

,block,start,end,inversion_conflict
0,"Block627,Block625",76379,76482,False
1,"Block627,Block626",76853,77129,False
2,"Block678,Block65",335018,335048,False
3,"Block1011,Block719",654048,654075,False
4,"Block913,Block96",1147977,1148009,False


In [35]:
genes = [i for i in inside_regions.values()]
overlap_start, overlap, overlap_end = [],[],[]
for i in range(len(genes)):
    overlap_start.append(len(genes[i][0]))
    overlap.append(len(genes[i][1]))
    overlap_end.append(len(genes[i][2]))

print(sum(overlap_start), sum(overlap), sum(overlap_end))
    

3 0 2


### With inversion conflict

In [36]:
block_region = ATCC_overlaps[ATCC_overlaps['inside_block']==True].reset_index(drop=True)
block_region = block_region[block_region['inversion_conflict']==True].reset_index(drop=True)

block_region['start'] = block_region['start_y']
block_region['end'] = block_region['end_y']
block_region['block'] = block_region['Overlapping_blocks']
block_region = block_region[['block', 'start', 'end', 'inversion_conflict']]

In [37]:
block_regions={}
for i in range(len(block_region)):
    block_regions[block_region['block'][i]] = overlapping_gene(block_region['start'][i], 
                                                                      block_region['end'][i], ATCC)
pd.DataFrame(block_regions)

,"Block637,Block609","Block637,Block515","Block644,Block1080","Block707,Block699","Block710,Block686","Block1029,Block701","Block1028,Block1020","Block1028,Block229","Block1023,Block152","Block1017,Block415",...,"Block1141,Block189","Block57,Block885","Block509,Block51","Block506,Block398","Block503,Block104","Block485,Block690","Block416,Block213","Block307,Block742","Block216,Block254","Block208,Block25"
0,[],[DR76_RS00735],[],[],[],[],[DR76_RS02575],[],[],[DR76_RS02875],...,[],[],[],[DR76_RS15375],[],[],[],[DR76_RS18985],[],[]
1,[],[],[],[],[],[],[],[],[],[],...,[],[DR76_RS14140],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],[],[],...,[],[DR76_RS14140],[],[],[],[],[],[],[],[]


In [38]:
block_region.head()

,block,start,end,inversion_conflict
0,"Block637,Block609",133257,133286,True
1,"Block637,Block515",139971,140002,True
2,"Block644,Block1080",168513,168548,True
3,"Block707,Block699",416208,416240,True
4,"Block710,Block686",464547,464580,True


In [39]:
genes = [i for i in block_regions.values()]
overlap_start, overlap, overlap_end = [],[],[]
for i in range(len(genes)):
    overlap_start.append(len(genes[i][0]))
    overlap.append(len(genes[i][1]))
    overlap_end.append(len(genes[i][2]))

print(sum(overlap_start), sum(overlap), sum(overlap_end))
    

17 1 1


In [40]:
ATCC.columns

Index(['gene_id', 'gbk_start', 'gbk_end', 'gbk_midpoint', 'gbk_gene_id',
       'gbk_old_locus_tag', 'gbk_strand', 'norm_exp'],
      dtype='object')

In [41]:
ATCC[ATCC['gene_id']=='DR76_RS06360']

,gene_id,gbk_start,gbk_end,gbk_midpoint,gbk_gene_id,gbk_old_locus_tag,gbk_strand,norm_exp
1234,DR76_RS06360,1331719,1332900,1332309,NaN,DR76_1229,1,4.471491


### Omitted blocks that are inside another block 

In [42]:
block_ends=[]
inside_block=[]
for i in range(len(ATCC_block)-1): 
    block_ends.append(ATCC_block.end[i])
    if not(all(ATCC_block.end[i+1] > x for x in block_ends)):
           inside_block.append(ATCC_block.block[i+1])

In [43]:
##View inside block df 
inside_block_df = pd.merge(ATCC_block, pd.DataFrame({'block':inside_block}), how='right', on='block')
print('there are', len(inside_block_df), 'inside blocks and', sum(inside_block_df['inversion']), 'are inverted')
ATCC_inside_inverted = inside_block_df[inside_block_df['inversion']==1]

there are 60 inside blocks and 22 are inverted


In [44]:
ATCC_block_final = ATCC_block[~ATCC_block['block'].isin(inside_block)]

In [45]:
len(ATCC_block)

1148

In [46]:
ATCC_block_final.head()

,block,taxa,start,end,rev_comp,inversion,block_order,diff,overlaps
0,Block611,CP009072,0,19237,0,0,1,0,False
1,Block612,CP009072,19639,25321,0,0,2,402,False
2,Block613,CP009072,25328,25469,0,0,3,7,False
3,Block614,CP009072,25805,30879,0,0,4,336,False
4,Block615,CP009072,30896,41163,0,0,5,17,False


In [47]:
ATCC_block_final['block'].nunique()

1088

## ATCC expression data                  

In [48]:
#matching blocks by start
ATCC_block_by_start = [match_block_start(i, ATCC_block_final) for i in ATCC.gbk_start]

In [49]:
#matching block by end
ATCC_block_by_end = [match_block_end(i, ATCC_block_final) for i in ATCC.gbk_end]

In [50]:
#add new columns and make it a new dataframe
ATCC_new = ATCC.copy()
ATCC_new['block_by_start'] = ATCC_block_by_start
ATCC_new['block_by_end'] = ATCC_block_by_end

#check if the start and end identifies the same block
ATCC_new['Single_block'] = ATCC_new['block_by_start'] == ATCC_new['block_by_end']
print('ATCC has', len(ATCC_new), 'rows and', sum(ATCC_new.Single_block), 'are single blocks')

ATCC has 4825 rows and 3181 are single blocks


In [51]:
#non single blocks - check for neighbouring 
ATCC_non_sb = ATCC_new[ATCC_new.Single_block==False]
print('There are',len(ATCC_non_sb), 'spitted genes')

#merging by the gene start 
ATCC_non_sb = pd.merge(ATCC_non_sb, ATCC_block_final, how='left', left_on='block_by_start', right_on='block')
ATCC_non_sb = ATCC_non_sb.drop(['Single_block','block', 'taxa', 'start', 'end', 'rev_comp','inversion','diff'],1)
ATCC_non_sb.columns.values[-2] = 'start_order'
ATCC_non_sb.columns.values[-1] = 'start_block_overlap'

#merging by the gene end 
ATCC_non_sb = pd.merge(ATCC_non_sb, ATCC_block_final, how='left', left_on='block_by_end', right_on='block')
ATCC_non_sb = ATCC_non_sb.drop(['block', 'taxa', 'start', 'end', 'rev_comp','inversion','diff'],1)
ATCC_non_sb.columns.values[-2] = 'end_order'
ATCC_non_sb.columns.values[-1] = 'end_block_overlap'

There are 1644 spitted genes


In [52]:
#check continuity, neighbour and overlapping blocks 
ATCC_non_sb['check_continuity'] = ATCC_non_sb['start_order'] < ATCC_non_sb['end_order']
print('There are', len(ATCC_non_sb), 'spitted genes,', sum(ATCC_non_sb['check_continuity']), 'are continous.') 
ATCC_non_sb['check_neighbour'] = ATCC_non_sb['start_order']+1 == ATCC_non_sb['end_order']
print('There are', len(ATCC_non_sb), 'spitted genes,', sum(ATCC_non_sb['check_neighbour']), 'are neighbouring blocks') 

There are 1644 spitted genes, 1644 are continous.
There are 1644 spitted genes, 1509 are neighbouring blocks


In [53]:
ATCC_non_sb = pd.merge(ATCC_non_sb, ATCC_block[['block','end']], left_on='block_by_start',
                      right_on='block', how='left')
ATCC_non_sb = pd.merge(ATCC_non_sb, ATCC_block[['block','start']], left_on='block_by_end',
                      right_on='block', how='left')

In [54]:
ATCC_non_sb['space_between_block'] = ATCC_non_sb['start'] - ATCC_non_sb['end']
ATCC_non_sb['space_between_block'].describe()

count     1644.000000
mean     14627.071168
std      17184.982068
min          1.000000
25%       1237.500000
50%       8394.000000
75%      20418.000000
max      67454.000000
Name: space_between_block, dtype: float64

In [55]:
ATCC_non_sb[['gbk_start', 'gbk_end','start',]]

,gbk_start,gbk_end,start
0,17656,21444,19639
1,25147,25463,25328
2,30599,32602,30896
3,41174,43471,41179
4,50926,51876,50938
...,...,...,...
1639,5114873,5116981,5119318
1640,5116992,5117708,5119318
1641,5117708,5118922,5119318
1642,5122984,5123763,5123461


## single block and merge with block file 

In [56]:
ATCC_sb = ATCC_new[ATCC_new['Single_block']==True]

In [57]:
ATCC_sb = pd.merge(ATCC_sb, ATCC_block_final, how='left', left_on='block_by_start', right_on='block')

In [58]:
print('There are', len(ATCC_sb), 'genes and', sum(ATCC_sb['overlaps']), 'are in overlapping block')

There are 3181 genes and 33 are in overlapping block


In [59]:
ATCC_sb_overlap = ATCC_sb[ATCC_sb['overlaps']==1]
sum(ATCC_sb_overlap['inversion'])

5

In [60]:
ATCC_sb['block'].nunique()

599

# BW25113

In [61]:
##find difference between blocks
diff = [0]
for i in range(len(BW25113_block.start)-1):
    diff.append(BW25113_block.start[i+1] - BW25113_block.end[i])

BW25113_block['diff'] = diff
BW25113_block['overlaps'] = BW25113_block['diff'] < 0 

print('There are', len(BW25113_block), 'blocks and', sum(BW25113_block['overlaps']), 'are overlapping blocks')
#this current block is starting inside the last block
#BW25113_block[BW25113_block['overlaps']==True] 
#BW25113_block.loc[15:20]

There are 1148 blocks and 5 are overlapping blocks


## Overlapping genes

In [62]:
# block = ['a','b','c','d']
# start = [1,6,10,21]
# end = [20,9,12,25]

block = BW25113_block['block']
start = BW25113_block['start']
end = BW25113_block['end']

N = len(block)

tally =[]
for i in range(len(block)):
    tally.append((start[i], block[i],"start"))
    tally.append((end[i], block[i],"end"))
    
tally = sorted(tally, key=lambda x:x[0])

groups={}
stack={}
for entry in tally:
    t = entry[0]
    name = entry[1]
    action = entry[2]
    
    if action == "start":
        stack[name]= True
        if len(stack)>1:
            groups[','.join(stack)] = True
    if action == "end":
        del stack[name]

#print(tally)
#print(groups)

In [63]:
BW25113_overlaps = pd.DataFrame({'Overlapping_blocks':[x for x in groups.keys()]})
blocks = BW25113_overlaps['Overlapping_blocks'].str.split(',',n=2, expand=True)
BW25113_overlaps['Bigblock'] = blocks[0]
BW25113_overlaps['Smallblock1'] = blocks[1]
BW25113_overlaps = pd.merge(BW25113_overlaps, BW25113_block[['block','start','end','inversion']], left_on='Bigblock',right_on='block', how='left')
BW25113_overlaps = pd.merge(BW25113_overlaps, BW25113_block[['block','start','end','inversion']], left_on='Smallblock1',right_on='block', how='left')
BW25113_overlaps['inversion_conflict'] = BW25113_overlaps['inversion_x'] != BW25113_overlaps['inversion_y']
BW25113_overlaps['inside_block'] = BW25113_overlaps['end_y'] < BW25113_overlaps['end_x']
print('there are', sum(BW25113_overlaps['inversion_conflict']), 'out of', len(BW25113_overlaps),'overlaps that have inconsistent inversion')
print(sum(BW25113_overlaps[BW25113_overlaps['inversion_conflict']]['inside_block']),'inconsitencies are inside blocks')
BW25113_overlaps.head()

there are 1 out of 5 overlaps that have inconsistent inversion
0 inconsitencies are inside blocks


,Overlapping_blocks,Bigblock,Smallblock1,block_x,start_x,end_x,inversion_x,block_y,start_y,end_y,inversion_y,inversion_conflict,inside_block
0,"Block56,Block57",Block56,Block57,Block56,220668,221215,0,Block57,220669,223414,0,False,False
1,"Block187,Block17",Block187,Block17,Block187,748625,753863,1,Block17,751912,751934,1,False,True
2,"Block501,Block97",Block501,Block97,Block501,1933474,1940335,1,Block97,1938486,1938513,1,False,True
3,"Block913,Block96",Block913,Block96,Block913,3585984,3597071,1,Block96,3591755,3591787,1,False,True
4,"Block1105,Block1106",Block1105,Block1106,Block1105,4460195,4460227,1,Block1106,4460214,4462437,0,True,False


In [64]:
overlapping_region = BW25113_overlaps[BW25113_overlaps['inside_block']==False].reset_index(drop=True)

overlapping_region['start'] = overlapping_region['start_y']
overlapping_region['end'] = overlapping_region['end_x']
overlapping_region['block'] = overlapping_region['Overlapping_blocks']
overlapping_region = overlapping_region[['block', 'start', 'end', 'inversion_conflict']]
overlapping_region 

,block,start,end,inversion_conflict
0,"Block56,Block57",220669,221215,False
1,"Block1105,Block1106",4460214,4460227,True


In [65]:
def overlapping_gene2(start, end, block_file):
    inside_gene, overlapping_start, overlapping_end=[],[],[]
    for i in range(len(block_file)):
        if (block_file['gbk_start'][i] < start and block_file['gbk_end'][i] > start):
            overlapping_start.append(block_file['Locus_tag'][i])
        if (block_file['gbk_start'][i] > start and block_file['gbk_end'][i] < end):
            inside_gene.append(block_file['Locus_tag'][i]) 
        if (block_file['gbk_start'][i] > start and block_file['gbk_start'][i] < end):
            overlapping_end.append(block_file['Locus_tag'][i]) 
    return [overlapping_start,inside_gene,overlapping_end]

In [66]:
BW25113.gbk_end = [i.replace('>','') for i in BW25113.gbk_end]
BW25113['gbk_end'] = pd.to_numeric(BW25113['gbk_end'])
BW25113_regions = {}
regions={}
for i in range(len(overlapping_region)):
    regions[overlapping_region['block'][i]] = overlapping_gene2(overlapping_region['start'][i], 
                                                               overlapping_region['end'][i], BW25113)
regions

{'Block56,Block57': [[], [], []], 'Block1105,Block1106': [[], [], []]}

## inside block genes check

### no inversion conflict

In [67]:
insideblock_region = BW25113_overlaps[BW25113_overlaps['inside_block']==True].reset_index(drop=True)
insideblock_region = insideblock_region[insideblock_region['inversion_conflict']==False].reset_index(drop=True)

insideblock_region['start'] = insideblock_region['start_y']
insideblock_region['end'] = insideblock_region['end_y']
insideblock_region['block'] = insideblock_region['Overlapping_blocks']
insideblock_region = insideblock_region[['block', 'start', 'end', 'inversion_conflict']]

In [68]:
inside_regions={}
for i in range(len(insideblock_region)):
    inside_regions[insideblock_region['block'][i]] = overlapping_gene2(insideblock_region['start'][i], 
                                                                      insideblock_region['end'][i], BW25113)
pd.DataFrame(inside_regions)

,"Block187,Block17","Block501,Block97","Block913,Block96"
0,[b0723],[],[]
1,[],[],[]
2,[],[],[]


In [69]:
insideblock_region.head()

,block,start,end,inversion_conflict
0,"Block187,Block17",751912,751934,False
1,"Block501,Block97",1938486,1938513,False
2,"Block913,Block96",3591755,3591787,False


In [70]:
genes = [i for i in inside_regions.values()]
overlap_start, overlap, overlap_end = [],[],[]
for i in range(len(genes)):
    overlap_start.append(len(genes[i][0]))
    overlap.append(len(genes[i][1]))
    overlap_end.append(len(genes[i][2]))

print(sum(overlap_start), sum(overlap), sum(overlap_end))
    

1 0 0


### With inversion conflict

In [71]:
block_region = BW25113_overlaps[BW25113_overlaps['inside_block']==True].reset_index(drop=True)
block_region = block_region[block_region['inversion_conflict']==True].reset_index(drop=True)

block_region['start'] = block_region['start_y']
block_region['end'] = block_region['end_y']
block_region['block'] = block_region['Overlapping_blocks']
block_region = block_region[['block', 'start', 'end', 'inversion_conflict']]

In [72]:
block_regions={}
for i in range(len(block_region)):
    block_regions[block_region['block'][i]] = overlapping_gene2(block_region['start'][i], 
                                                                      block_region['end'][i], BW25113)
pd.DataFrame(block_regions)

""


In [73]:
block_region.head()

,block,start,end,inversion_conflict


In [74]:
genes = [i for i in block_regions.values()]
overlap_start, overlap, overlap_end = [],[],[]
for i in range(len(genes)):
    overlap_start.append(len(genes[i][0]))
    overlap.append(len(genes[i][1]))
    overlap_end.append(len(genes[i][2]))

print(sum(overlap_start), sum(overlap), sum(overlap_end))
    

0 0 0


### Omitted blocks that are inside another block 

In [75]:
block_ends=[]
inside_block=[]
for i in range(len(BW25113_block)-1): 
    block_ends.append(BW25113_block.end[i])
    if not(all(BW25113_block.end[i+1] > x for x in block_ends)):
           inside_block.append(BW25113_block.block[i+1])

print('There are',len(inside_block), 'inside blocks')

There are 3 inside blocks


In [76]:
##View inside block df 
inside_block_df = pd.merge(BW25113_block, pd.DataFrame({'block':inside_block}), how='right', on='block')
inside_block_df.head()
print('there are', len(inside_block_df), 'inside blocks and', sum(inside_block_df['inversion']), 'are inverted')
BW25113_inside_inverted = inside_block_df[inside_block_df['inversion']==1]
BW25113_inside_inverted

there are 3 inside blocks and 3 are inverted


,block,taxa,start,end,rev_comp,inversion,block_order,diff,overlaps
0,Block17,NZ_CP009273,751912,751934,1,1,185,-1951,True
1,Block97,NZ_CP009273,1938486,1938513,1,1,500,-1849,True
2,Block96,NZ_CP009273,3591755,3591787,1,1,913,-5316,True


In [77]:
BW25113_block_final = BW25113_block[~BW25113_block['block'].isin(inside_block)]

In [78]:
print('there are', sum(BW25113_block_final['overlaps']), 'overlapping blocks and', 
                      sum(pd.DataFrame(BW25113_block_final[BW25113_block_final['overlaps']==1])['inversion']), 'are inverted')

there are 2 overlapping blocks and 0 are inverted


In [79]:
BW25113_overlapped = pd.DataFrame(BW25113_block_final[BW25113_block_final['overlaps']==1])
BW25113_overlapped

,block,taxa,start,end,rev_comp,inversion,block_order,diff,overlaps
55,Block57,NZ_CP009273,220669,223414,0,0,56,-546,True
1105,Block1106,NZ_CP009273,4460214,4462437,0,0,1106,-13,True


In [80]:
BW25113_block_final['block'].nunique()

1145

## BW expression data 

In [81]:
#matching blocks by start
BW25113_block_by_start = [match_block_start(i, BW25113_block_final) for i in BW25113.gbk_start]

In [82]:
print(max(BW25113_block_final['end']),max(BW25113['gbk_end']))

4631461 4631445


In [83]:
#matching block by end
BW25113_block_by_end = [match_block_end(i, BW25113_block_final) for i in BW25113.gbk_end]

In [84]:
#add new columns and make it a new dataframe
BW25113_new = BW25113.copy()
BW25113_new['block_by_start'] = BW25113_block_by_start
BW25113_new['block_by_end'] = BW25113_block_by_end

#check if the start and end identifies the same block
BW25113_new['Single_block'] = BW25113_new['block_by_start'] == BW25113_new['block_by_end']
print('BW25113 has', len(BW25113_new), 'rows and', sum(BW25113_new.Single_block), 'are single blocks')

BW25113 has 4077 rows and 3040 are single blocks


In [85]:
#non single blocks - check for neighbouring 
BW25113_non_sb = BW25113_new[BW25113_new.Single_block==False]
print('There are',len(BW25113_non_sb), 'spitted genes')

#merging by the gene start 
BW25113_non_sb = pd.merge(BW25113_non_sb, BW25113_block_final, how='left', left_on='block_by_start', right_on='block')
BW25113_non_sb = BW25113_non_sb.drop(['Single_block','block', 'taxa', 'start', 'end', 'rev_comp','inversion','diff'],1)
BW25113_non_sb.columns.values[-2] = 'start_order'
BW25113_non_sb.columns.values[-1] = 'start_block_overlap'

#merging by the gene end 
BW25113_non_sb = pd.merge(BW25113_non_sb, BW25113_block_final, how='left', left_on='block_by_end', right_on='block')
BW25113_non_sb = BW25113_non_sb.drop(['block', 'taxa', 'start', 'end', 'rev_comp','inversion','diff'],1)
BW25113_non_sb.columns.values[-2] = 'end_order'
BW25113_non_sb.columns.values[-1] = 'end_block_overlap'

There are 1037 spitted genes


In [86]:
#check continuity, neighbour and overlapping blocks 
BW25113_non_sb['check_continuity'] = BW25113_non_sb['start_order'] < BW25113_non_sb['end_order']
print('There are', len(BW25113_non_sb), 'spitted genes,', sum(BW25113_non_sb['check_continuity']), 'are continous.') 
BW25113_non_sb['check_neighbour'] = BW25113_non_sb['start_order']+1 == BW25113_non_sb['end_order']
print('There are', len(BW25113_non_sb), 'spitted genes,', sum(BW25113_non_sb['check_neighbour']), 'are neighbouring blocks') 

There are 1037 spitted genes, 1037 are continous.
There are 1037 spitted genes, 954 are neighbouring blocks


In [87]:
BW25113_non_sb = pd.merge(BW25113_non_sb, BW25113_block[['block','end']], left_on='block_by_start',
                      right_on='block', how='left')
BW25113_non_sb = pd.merge(BW25113_non_sb, BW25113_block[['block','start']], left_on='block_by_end',
                      right_on='block', how='left')

In [88]:
BW25113_non_sb['space_between_block'] = BW25113_non_sb['start'] - BW25113_non_sb['end']

In [89]:
pd.DataFrame(BW25113_non_sb[BW25113_non_sb['space_between_block']>0])['space_between_block'].describe()

count      1037.000000
mean      17734.105111
std       34112.310906
min           1.000000
25%         381.000000
50%        3070.000000
75%       11926.000000
max      114522.000000
Name: space_between_block, dtype: float64

In [90]:
BW25113_non_sb[BW25113_non_sb['space_between_block']<0]

,Locus_tag,gbk_locus_tag,gbk_start,gbk_end,gbk_midpoint,gbk_gene_id,gbk_old_locus_tag,gbk_strand,norm_exp,block_by_start,...,start_block_overlap,end_order,end_block_overlap,check_continuity,check_neighbour,block_x,end,block_y,start,space_between_block


## single block and merge with block file 

In [91]:
BW25113_sb = BW25113_new[BW25113_new['Single_block']==True]
BW25113_sb = pd.merge(BW25113_sb, BW25113_block, how='left', left_on='block_by_start', right_on='block')
print('There are', len(BW25113_sb), 'single block genes')

There are 3040 single block genes


In [92]:
print('There are', len(BW25113_sb), 'genes and', sum(BW25113_sb['overlaps']), 'are in overlapping block')

There are 3040 genes and 4 are in overlapping block


In [93]:
BW25113_sb['block'].nunique()

577

# K12DH 

### block file - spaces between blocks

In [94]:
##find difference between blocks
diff = [0]
for i in range(len(K12DH_block.start)-1):
    diff.append(K12DH_block.start[i+1] - K12DH_block.end[i])

K12DH_block['diff'] = diff
K12DH_block['overlaps'] = K12DH_block['diff'] < 0 

print('There are', len(K12DH_block), 'blocks and', sum(K12DH_block['overlaps']), 'are overlapping blocks')
#this current block is starting inside the last block
#K12DH_block[K12DH_block['overlaps']==True] 
#K12DH_block.loc[15:20]

There are 1148 blocks and 5 are overlapping blocks


## Overlapping genes

In [95]:
# block = ['a','b','c','d']
# start = [1,6,10,21]
# end = [20,9,12,25]

block = K12DH_block['block']
start = K12DH_block['start']
end = K12DH_block['end']

N = len(block)

tally =[]
for i in range(len(block)):
    tally.append((start[i], block[i],"start"))
    tally.append((end[i], block[i],"end"))
    
tally = sorted(tally, key=lambda x:x[0])

groups={}
stack={}
for entry in tally:
    t = entry[0]
    name = entry[1]
    action = entry[2]
    
    if action == "start":
        stack[name]= True
        if len(stack)>1:
            groups[','.join(stack)] = True
    if action == "end":
        del stack[name]

#print(tally)
#print(groups)

In [96]:
K12DH_overlaps = pd.DataFrame({'Overlapping_blocks':[x for x in groups.keys()]})
blocks = K12DH_overlaps['Overlapping_blocks'].str.split(',',n=2, expand=True)
K12DH_overlaps['Bigblock'] = blocks[0]
K12DH_overlaps['Smallblock1'] = blocks[1]
K12DH_overlaps = pd.merge(K12DH_overlaps, K12DH_block[['block','start','end','inversion']], left_on='Bigblock',right_on='block', how='left')
K12DH_overlaps = pd.merge(K12DH_overlaps, K12DH_block[['block','start','end','inversion']], left_on='Smallblock1',right_on='block', how='left')
K12DH_overlaps['inversion_conflict'] = K12DH_overlaps['inversion_x'] != K12DH_overlaps['inversion_y']
K12DH_overlaps['inside_block'] = K12DH_overlaps['end_y'] < K12DH_overlaps['end_x']
print('there are', sum(K12DH_overlaps['inversion_conflict']), 'out of', len(K12DH_overlaps),'overlaps that have inconsistent inversion')
print(sum(K12DH_overlaps[K12DH_overlaps['inversion_conflict']]['inside_block']),'inconsitencies are inside blocks')
K12DH_overlaps.head()

there are 1 out of 5 overlaps that have inconsistent inversion
0 inconsitencies are inside blocks


,Overlapping_blocks,Bigblock,Smallblock1,block_x,start_x,end_x,inversion_x,block_y,start_y,end_y,inversion_y,inversion_conflict,inside_block
0,"Block56,Block57",Block56,Block57,Block56,198285,198832,0,Block57,198286,201031,0,False,False
1,"Block187,Block17",Block187,Block17,Block187,804984,810222,1,Block17,808271,808293,1,False,True
2,"Block501,Block97",Block501,Block97,Block501,2027812,2034673,1,Block97,2032824,2032851,1,False,True
3,"Block913,Block96",Block913,Block96,Block913,3688392,3699479,1,Block96,3694163,3694195,1,False,True
4,"Block1105,Block1106",Block1105,Block1106,Block1105,4570209,4570241,1,Block1106,4570228,4572451,0,True,False


In [97]:
overlapping_region = K12DH_overlaps[K12DH_overlaps['inside_block']==False].reset_index(drop=True)

overlapping_region['start'] = overlapping_region['start_y']
overlapping_region['end'] = overlapping_region['end_x']
overlapping_region['block'] = overlapping_region['Overlapping_blocks']
overlapping_region = overlapping_region[['block', 'start', 'end', 'inversion_conflict']]
overlapping_region 

,block,start,end,inversion_conflict
0,"Block56,Block57",198286,198832,False
1,"Block1105,Block1106",4570228,4570241,True


In [98]:
K12DH['gbk_end'] = K12DH['gbk_end'].apply(lambda x:x.replace('>',''))

In [99]:
K12DH['gbk_end'] = pd.to_numeric(K12DH['gbk_end'])

In [100]:
def overlapping_gene3(start, end, block_file):
    inside_gene, overlapping_start, overlapping_end=[],[],[]
    for i in range(len(block_file)):
        if (block_file['gbk_start'][i] < start and block_file['gbk_end'][i] > start):
            overlapping_start.append(block_file['locus_tag'][i])
        if (block_file['gbk_start'][i] > start and block_file['gbk_end'][i] < end):
            inside_gene.append(block_file['locus_tag'][i]) 
        if (block_file['gbk_start'][i] > start and block_file['gbk_start'][i] < end):
            overlapping_end.append(block_file['locus_tag'][i]) 
    return [overlapping_start,inside_gene,overlapping_end]

In [101]:
K12DH_regions = {}
regions={}
for i in range(len(overlapping_region)):
    regions[overlapping_region['block'][i]] = overlapping_gene3(overlapping_region['start'][i], 
                                                               overlapping_region['end'][i], K12DH)
regions

{'Block56,Block57': [['ECDH10B_0182'], [], []],
 'Block1105,Block1106': [[], [], []]}

## inside block genes check

### no inversion conflict

In [102]:
insideblock_region = K12DH_overlaps[K12DH_overlaps['inside_block']==True].reset_index(drop=True)
insideblock_region = insideblock_region[insideblock_region['inversion_conflict']==False].reset_index(drop=True)

insideblock_region['start'] = insideblock_region['start_y']
insideblock_region['end'] = insideblock_region['end_y']
insideblock_region['block'] = insideblock_region['Overlapping_blocks']
insideblock_region = insideblock_region[['block', 'start', 'end', 'inversion_conflict']]

In [103]:
inside_regions={}
for i in range(len(insideblock_region)):
    inside_regions[insideblock_region['block'][i]] = overlapping_gene3(insideblock_region['start'][i], 
                                                                      insideblock_region['end'][i], K12DH)
pd.DataFrame(inside_regions)

,"Block187,Block17","Block501,Block97","Block913,Block96"
0,[ECDH10B_0790],[],[]
1,[],[],[]
2,[],[],[]


In [104]:
insideblock_region.head()

,block,start,end,inversion_conflict
0,"Block187,Block17",808271,808293,False
1,"Block501,Block97",2032824,2032851,False
2,"Block913,Block96",3694163,3694195,False


In [105]:
genes = [i for i in inside_regions.values()]
overlap_start, overlap, overlap_end = [],[],[]
for i in range(len(genes)):
    overlap_start.append(len(genes[i][0]))
    overlap.append(len(genes[i][1]))
    overlap_end.append(len(genes[i][2]))

print(sum(overlap_start), sum(overlap), sum(overlap_end))
    

1 0 0


### With inversion conflict

In [106]:
block_region = K12DH_overlaps[K12DH_overlaps['inside_block']==True].reset_index(drop=True)
block_region = block_region[block_region['inversion_conflict']==True].reset_index(drop=True)

block_region['start'] = block_region['start_y']
block_region['end'] = block_region['end_y']
block_region['block'] = block_region['Overlapping_blocks']
block_region = block_region[['block', 'start', 'end', 'inversion_conflict']]

In [107]:
block_regions={}
for i in range(len(block_region)):
    block_regions[block_region['block'][i]] = overlapping_gene3(block_region['start'][i], 
                                                                      block_region['end'][i], K12DH)
pd.DataFrame(block_regions)

""


In [108]:
block_region.head()

,block,start,end,inversion_conflict


In [109]:
genes = [i for i in block_regions.values()]
overlap_start, overlap, overlap_end = [],[],[]
for i in range(len(genes)):
    overlap_start.append(len(genes[i][0]))
    overlap.append(len(genes[i][1]))
    overlap_end.append(len(genes[i][2]))

print(sum(overlap_start), sum(overlap), sum(overlap_end))
    

0 0 0


### Omitted blocks that are inside another block 

In [110]:
block_ends=[]
inside_block=[]
for i in range(len(K12DH_block)-1): 
    block_ends.append(K12DH_block.end[i])
    if not(all(K12DH_block.end[i+1] > x for x in block_ends)):
           inside_block.append(K12DH_block.block[i+1])

print('There are',len(inside_block), 'inside blocks')

There are 3 inside blocks


In [111]:
##View inside block df 
inside_block_df = pd.merge(K12DH_block, pd.DataFrame({'block':inside_block}), how='right', on='block')
inside_block_df.head()
print('there are', len(inside_block_df), 'inside blocks and', sum(inside_block_df['inversion']), 'are inverted')
K12DH_inside_inverted = inside_block_df[inside_block_df['inversion']==1]
K12DH_inside_inverted

there are 3 inside blocks and 3 are inverted


,block,taxa,start,end,rev_comp,inversion,block_order,diff,overlaps
0,Block17,NC_010473,808271,808293,1,1,183,-1951,True
1,Block97,NC_010473,2032824,2032851,1,1,500,-1849,True
2,Block96,NC_010473,3694163,3694195,1,1,913,-5316,True


In [112]:
K12DH_block_final = K12DH_block[~K12DH_block['block'].isin(inside_block)]

In [113]:
print('there are', sum(K12DH_block_final['overlaps']), 'overlapping blocks and', 
                      sum(pd.DataFrame(K12DH_block_final[K12DH_block_final['overlaps']==1])['inversion']), 'are inverted')

there are 2 overlapping blocks and 0 are inverted


In [114]:
K12DH_overlapped = pd.DataFrame(K12DH_block_final[K12DH_block_final['overlaps']==1])
K12DH_overlapped

,block,taxa,start,end,rev_comp,inversion,block_order,diff,overlaps
55,Block57,NC_010473,198286,201031,0,0,56,-546,True
1105,Block1106,NC_010473,4570228,4572451,0,0,1106,-13,True


## K12 DH Expression data 

In [115]:
#matching blocks by start
K12DH_block_by_start = [match_block_start(i, K12DH_block_final) for i in K12DH.gbk_start]

In [116]:
#matching block by end
K12DH_block_by_end = [match_block_end(i, K12DH_block_final) for i in K12DH.gbk_end]

In [117]:
#add new columns and make it a new dataframe
K12DH_new = K12DH
K12DH_new['block_by_start'] = K12DH_block_by_start
K12DH_new['block_by_end'] = K12DH_block_by_end

#check if the start and end identifies the same block
K12DH_new['Single_block'] = K12DH_new['block_by_start'] == K12DH_new['block_by_end']
print('K12DH has', len(K12DH_new), 'rows and', sum(K12DH_new.Single_block), 'are single blocks')

K12DH has 4247 rows and 3058 are single blocks


In [118]:
#non single blocks - check for neighbouring 
K12DH_non_sb = K12DH_new[K12DH_new.Single_block==False]
print('There are',len(K12DH_non_sb), 'spitted genes')

#merging by the gene start 
K12DH_non_sb = pd.merge(K12DH_non_sb, K12DH_block, how='left', left_on='block_by_start', right_on='block')
K12DH_non_sb = K12DH_non_sb.drop(['Single_block','block', 'taxa', 'start', 'end', 'rev_comp','inversion','diff'],1)
K12DH_non_sb.columns.values[-2] = 'start_order'
K12DH_non_sb.columns.values[-1] = 'start_block_overlap'

#merging by the gene end 
K12DH_non_sb = pd.merge(K12DH_non_sb, K12DH_block, how='left', left_on='block_by_end', right_on='block')
K12DH_non_sb = K12DH_non_sb.drop(['block', 'taxa', 'start', 'end', 'rev_comp','inversion','diff'],1)
K12DH_non_sb.columns.values[-2] = 'end_order'
K12DH_non_sb.columns.values[-1] = 'end_block_overlap'

There are 1189 spitted genes


In [119]:
#check continuity, neighbour and overlapping blocks 
K12DH_non_sb['check_continuity'] = K12DH_non_sb['start_order'] < K12DH_non_sb['end_order']
print('There are', len(K12DH_non_sb), 'spitted genes,', sum(K12DH_non_sb['check_continuity']), 'are continous.') 
K12DH_non_sb['check_neighbour'] = K12DH_non_sb['start_order']+1 == K12DH_non_sb['end_order']
print('There are', len(K12DH_non_sb), 'spitted genes,', sum(K12DH_non_sb['check_neighbour']), 'are neighbouring blocks') 

There are 1189 spitted genes, 1189 are continous.
There are 1189 spitted genes, 1094 are neighbouring blocks


In [120]:
K12DH_non_sb = pd.merge(K12DH_non_sb, K12DH_block[['block','end']], left_on='block_by_start',
                      right_on='block', how='left')
K12DH_non_sb = pd.merge(K12DH_non_sb, K12DH_block[['block','start']], left_on='block_by_end',
                      right_on='block', how='left')

In [121]:
K12DH_non_sb['space_between_block'] = K12DH_non_sb['start'] - K12DH_non_sb['end']
K12DH_non_sb['space_between_block'].describe()

count      1189.000000
mean      44080.492010
std       85125.201727
min           1.000000
25%         465.000000
50%        3493.000000
75%       13842.000000
max      227782.000000
Name: space_between_block, dtype: float64

## single block and merge with block file 

In [122]:
K12DH_sb = K12DH_new[K12DH_new['Single_block']==True]
K12DH_sb = pd.merge(K12DH_sb, K12DH_block, how='left', left_on='block_by_start', right_on='block')
print('There are', len(K12DH_sb), 'single block genes')

There are 3058 single block genes


In [123]:
print('There are', len(K12DH_sb), 'genes and', sum(K12DH_sb['overlaps']), 'are in overlapping block')

There are 3058 genes and 6 are in overlapping block


In [124]:
K12DH_sb['block'].nunique()  

594

# K12MG expression data 

### block file - spaces between blocks

In [125]:
##find difference between blocks
diff = [0]
for i in range(len(K12MG_block.start)-1):
    diff.append(K12MG_block.start[i+1] - K12MG_block.end[i])

K12MG_block['diff'] = diff
K12MG_block['overlaps'] = K12MG_block['diff'] < 0 

print('There are', len(K12MG_block), 'blocks and', sum(K12MG_block['overlaps']), 'are overlapping blocks')
#this current block is starting inside the last block
#K12MG_block[K12MG_block['overlaps']==True] 
#K12MG_block.loc[15:20]

There are 1148 blocks and 2 are overlapping blocks


In [126]:
print('there are', sum(K12MG_block['overlaps']), 'overlapping blocks and', 
                      sum(pd.DataFrame(K12MG_block[K12MG_block['overlaps']==1])['inversion']), 'are inverted')

there are 2 overlapping blocks and 0 are inverted


In [127]:
K12MG_overlapped = pd.DataFrame(K12MG_block[K12MG_block['overlaps']==1])
K12MG_overlapped

,block,taxa,start,end,rev_comp,inversion,block_order,diff,overlaps
56,Block57,U00096000,224182,226927,0,0,57,-546,True
1105,Block1106,U00096000,4470397,4472620,0,0,1106,-13,True


In [128]:
# block = ['a','b','c','d']
# start = [1,6,10,21]
# end = [20,9,12,25]

block = K12MG_block['block']
start = K12MG_block['start']
end = K12MG_block['end']

N = len(block)

tally =[]
for i in range(len(block)):
    tally.append((start[i], block[i],"start"))
    tally.append((end[i], block[i],"end"))
    
tally = sorted(tally, key=lambda x:x[0])

groups={}
stack={}
for entry in tally:
    t = entry[0]
    name = entry[1]
    action = entry[2]
    
    if action == "start":
        stack[name]= True
        if len(stack)>1:
            groups[','.join(stack)] = True
    if action == "end":
        del stack[name]

#print(tally)
#print(groups)

In [129]:
K12MG_overlaps = pd.DataFrame({'Overlapping_blocks':[x for x in groups.keys()]})
blocks = K12MG_overlaps['Overlapping_blocks'].str.split(',',n=2, expand=True)
K12MG_overlaps['Bigblock'] = blocks[0]
K12MG_overlaps['Smallblock1'] = blocks[1]
K12MG_overlaps = pd.merge(K12MG_overlaps, K12MG_block[['block','start','end','inversion']], left_on='Bigblock',right_on='block', how='left')
K12MG_overlaps = pd.merge(K12MG_overlaps, K12MG_block[['block','start','end','inversion']], left_on='Smallblock1',right_on='block', how='left')
K12MG_overlaps['inversion_conflict'] = K12MG_overlaps['inversion_x'] != K12MG_overlaps['inversion_y']
K12MG_overlaps['inside_block'] = K12MG_overlaps['end_y'] < K12MG_overlaps['end_x']
print('there are', sum(K12MG_overlaps['inversion_conflict']), 'out of', len(K12MG_overlaps),'overlaps that have inconsistent inversion')
print(sum(K12MG_overlaps[K12MG_overlaps['inversion_conflict']]['inside_block']),'inconsitencies are inside blocks')
K12MG_overlaps.head()

there are 1 out of 2 overlaps that have inconsistent inversion
0 inconsitencies are inside blocks


,Overlapping_blocks,Bigblock,Smallblock1,block_x,start_x,end_x,inversion_x,block_y,start_y,end_y,inversion_y,inversion_conflict,inside_block
0,"Block56,Block57",Block56,Block57,Block56,224181,224728,0,Block57,224182,226927,0,False,False
1,"Block1105,Block1106",Block1105,Block1106,Block1105,4470378,4470410,1,Block1106,4470397,4472620,0,True,False


In [130]:
overlapping_region = K12MG_overlaps[K12MG_overlaps['inside_block']==False].reset_index(drop=True)

overlapping_region['start'] = overlapping_region['start_y']
overlapping_region['end'] = overlapping_region['end_x']
overlapping_region['block'] = overlapping_region['Overlapping_blocks']
overlapping_region = overlapping_region[['block', 'start', 'end', 'inversion_conflict']]
overlapping_region 

,block,start,end,inversion_conflict
0,"Block56,Block57",224182,224728,False
1,"Block1105,Block1106",4470397,4470410,True


In [131]:
K12MG_regions = {}
regions={}
for i in range(len(overlapping_region)):
    regions[overlapping_region['block'][i]] = overlapping_gene(overlapping_region['start'][i], 
                                                               overlapping_region['end'][i], K12MG)
regions

{'Block56,Block57': [[], [], []], 'Block1105,Block1106': [[], [], []]}

In [132]:
K12MG[K12MG['gene_id']=='rrsH']

,Locus_tag,gene_id,gbk_start,gbk_end,gbk_midpoint,gbk_gene_id,gbk_old_locus_tag,gbk_strand,norm_exp


### Omitted blocks that are inside another block 

In [133]:
block_ends=[]
inside_block=[]
for i in range(len(K12MG_block)-1): 
    block_ends.append(K12MG_block.end[i])
    if not(all(K12MG_block.end[i+1] > x for x in block_ends)):
           inside_block.append(K12MG_block.block[i+1])

print('There are',len(inside_block), 'inside blocks')

There are 0 inside blocks


In [134]:
K12MG_block['block'].nunique()

1148

## K12MG Expression data

In [135]:
#matching blocks by start
K12MG_block_by_start = [match_block_start(i, K12MG_block) for i in K12MG.gbk_start]

In [136]:
#matching block by end
K12MG_block_by_end = [match_block_end(i, K12MG_block) for i in K12MG.gbk_end]

In [137]:
#add new columns and make it a new dataframe
K12MG_new = K12MG
K12MG_new['block_by_start'] = K12MG_block_by_start
K12MG_new['block_by_end'] = K12MG_block_by_end

#check if the start and end identifies the same block
K12MG_new['Single_block'] = K12MG_new['block_by_start'] == K12MG_new['block_by_end']
print('K12MG has', len(K12MG_new), 'rows and', sum(K12MG_new.Single_block), 'are single blocks')

K12MG has 3891 rows and 2969 are single blocks


In [138]:
#non single blocks - check for neighbouring 
K12MG_non_sb = K12MG_new[K12MG_new.Single_block==False]
print('There are',len(K12MG_non_sb), 'spitted genes')

#merging by the gene start 
K12MG_non_sb = pd.merge(K12MG_non_sb, K12MG_block, how='left', left_on='block_by_start', right_on='block')
K12MG_non_sb = K12MG_non_sb.drop(['Single_block','block', 'taxa', 'start', 'end', 'rev_comp','inversion','diff'],1)
K12MG_non_sb.columns.values[-2] = 'start_order'
K12MG_non_sb.columns.values[-1] = 'start_block_overlap'

#merging by the gene end 
K12MG_non_sb = pd.merge(K12MG_non_sb, K12MG_block, how='left', left_on='block_by_end', right_on='block')
K12MG_non_sb = K12MG_non_sb.drop(['block', 'taxa', 'start', 'end', 'rev_comp','inversion','diff'],1)
K12MG_non_sb.columns.values[-2] = 'end_order'
K12MG_non_sb.columns.values[-1] = 'end_block_overlap'

There are 922 spitted genes


In [139]:
#check continuity, neighbour and overlapping blocks 
K12MG_non_sb['check_continuity'] = K12MG_non_sb['start_order'] < K12MG_non_sb['end_order']
print('There are', len(K12MG_non_sb), 'spitted genes,', sum(K12MG_non_sb['check_continuity']), 'are continous.') 
K12MG_non_sb['check_neighbour'] = K12MG_non_sb['start_order']+1 == K12MG_non_sb['end_order']
print('There are', len(K12MG_non_sb), 'spitted genes,', sum(K12MG_non_sb['check_neighbour']), 'are neighbouring blocks') 

There are 922 spitted genes, 922 are continous.
There are 922 spitted genes, 842 are neighbouring blocks


In [140]:
K12MG_non_sb = pd.merge(K12MG_non_sb, K12MG_block[['block','end']], left_on='block_by_start',
                      right_on='block', how='left')
K12MG_non_sb = pd.merge(K12MG_non_sb, K12MG_block[['block','start']], left_on='block_by_end',
                      right_on='block', how='left')

In [141]:
K12MG_non_sb['space_between_block'] = K12MG_non_sb['start'] - K12MG_non_sb['end']
K12MG_non_sb['space_between_block'].describe()

count       922.000000
mean      17646.595445
std       35228.601285
min           1.000000
25%         250.250000
50%        2069.000000
75%        9745.000000
max      114522.000000
Name: space_between_block, dtype: float64

## Non neighbouring blocks check

In [142]:
all_check = pd.DataFrame(K12MG_non_sb.block_by_start.value_counts())
nb_check = pd.DataFrame(K12MG_non_sb[K12MG_non_sb['check_neighbour']==True].block_by_end.value_counts())
non_nb_check = pd.DataFrame(K12MG_non_sb[K12MG_non_sb['check_neighbour']==False].block_by_end.value_counts())

## single block and merge with block file 

In [143]:
K12MG_sb = K12MG_new[K12MG_new['Single_block']==True]
K12MG_sb = pd.merge(K12MG_sb, K12MG_block, how='left', left_on='block_by_start', right_on='block')
print('There are', len(K12MG_sb), 'single block genes')

There are 2969 single block genes


In [144]:
print('There are', len(K12MG_sb), 'genes and', sum(K12MG_sb['overlaps']), 'are in overlapping block')

There are 2969 genes and 4 are in overlapping block


In [145]:
K12MG_sb['block'].nunique()

563

## Block analysis

### length of the block

In [146]:
ATCC_block_final['length'] = ATCC_block_final['end'] - ATCC_block_final['start']
ATCC_block_final['length'].describe()

count     1088.000000
mean      3444.801471
std       5002.603361
min         21.000000
25%         98.000000
50%       1307.000000
75%       4815.250000
max      38568.000000
Name: length, dtype: float64

In [147]:
BW25113_block_final['length'] = BW25113_block_final['end'] - BW25113_block_final['start']
BW25113_block_final['length'].describe()

count     1145.000000
mean      3275.960699
std       4932.066595
min         21.000000
25%         68.000000
50%       1084.000000
75%       4635.000000
max      38569.000000
Name: length, dtype: float64

In [148]:
K12DH_block_final['length'] = K12DH_block_final['end'] - K12DH_block_final['start']
K12DH_block_final['length'].describe()

count     1145.000000
mean      3275.952838
std       4932.048535
min         21.000000
25%         68.000000
50%       1084.000000
75%       4635.000000
max      38569.000000
Name: length, dtype: float64

In [149]:
K12MG_block['length'] = K12MG_block['end'] - K12MG_block['start']
K12MG_block['length'].describe()

count     1148.000000
mean      3267.469512
std       4928.405578
min         21.000000
25%         65.750000
50%       1071.000000
75%       4631.250000
max      38569.000000
Name: length, dtype: float64

In [150]:
test = pd.merge(BW25113_block_final, K12DH_block_final, on='block')
print(sum(test['start_x']==test['start_y']))
print(sum(test['end_x']==test['end_y']))
print(sum(test['rev_comp_x']==test['rev_comp_y']))
print(sum(test['inversion_x']==test['inversion_y']))

16
16
1141
1145


In [151]:
test2 = pd.merge(K12DH_block_final, K12MG_block, on='block')
print(sum(test2['start_x']==test2['start_y']))
print(sum(test2['end_x']==test2['end_y']))
print(sum(test2['rev_comp_x']==test2['rev_comp_y']))
print(sum(test2['inversion_x']==test2['inversion_y']))

16
16
1138
1145


In [152]:
test3 = pd.merge(BW25113_block_final, K12MG_block, on='block')
print(sum(test3['start_x']==test3['start_y']))
print(sum(test3['end_x']==test3['end_y']))
print(sum(test3['rev_comp_x']==test3['rev_comp_y']))
print(sum(test3['inversion_x']==test3['inversion_y']))

16
16
1142
1145


### Space between blocks

In [153]:
ATCC_block_final['diff'].describe()

count     1088.000000
mean      1474.011029
std       5076.324827
min       -203.000000
25%         19.750000
50%         66.000000
75%        303.250000
max      67454.000000
Name: diff, dtype: float64

In [154]:
BW25113_block_final['diff'].describe()

count      1145.000000
mean        776.874236
std        4249.358170
min        -546.000000
25%          11.000000
50%          52.000000
75%         230.000000
max      114522.000000
Name: diff, dtype: float64

In [155]:
K12DH_block_final['diff'].describe()

count      1145.000000
mean        824.627074
std        6998.438207
min        -546.000000
25%          11.000000
50%          54.000000
75%         247.000000
max      227782.000000
Name: diff, dtype: float64

In [156]:
K12MG_block['diff'].describe()

count      1148.000000
mean        775.774390
std        4213.000846
min        -546.000000
25%          11.000000
50%          53.000000
75%         238.750000
max      114522.000000
Name: diff, dtype: float64

### Space between blocks (non-overlapping)

In [157]:
pd.DataFrame(ATCC_block_final[ATCC_block_final['overlaps']==0])['diff'].describe()

count     1080.000000
mean      1485.671296
std       5093.288836
min          0.000000
25%         20.000000
50%         67.500000
75%        306.250000
max      67454.000000
Name: diff, dtype: float64

In [158]:
pd.DataFrame(BW25113_block_final[BW25113_block_final['overlaps']==0])['diff'].describe()

count      1143.000000
mean        778.722660
std        4252.832734
min           0.000000
25%          11.000000
50%          52.000000
75%         230.000000
max      114522.000000
Name: diff, dtype: float64

In [159]:
pd.DataFrame(K12DH_block_final[K12DH_block_final['overlaps']==0])['diff'].describe()

count      1143.000000
mean        826.559055
std        7004.402207
min           0.000000
25%          12.000000
50%          54.000000
75%         248.500000
max      227782.000000
Name: diff, dtype: float64

In [160]:
pd.DataFrame(K12MG_block[K12MG_block['overlaps']==0])['diff'].describe()

count      1146.000000
mean        777.616056
std        4216.432947
min           0.000000
25%          11.000000
50%          53.000000
75%         240.250000
max      114522.000000
Name: diff, dtype: float64

## Sample dataframe

In [161]:
ATCC_final = ATCC_sb.copy()
ATCC_final['locus_tag'] = np.nan
ATCC_final.head()

BW25113_sb = BW25113_sb.rename(columns={'Locus_tag': 'locus_tag'})

BW25113_final = BW25113_sb.copy()
BW25113_final['gene_id'] = np.nan
BW25113_final.head()

K12DH_sb.head()

K12MG_sb = K12MG_sb.rename(columns={'Locus_tag': 'locus_tag'})
K12MG_sb.head()

K12MG_sb.columns

K12MG_sb['strain'] = 'K12MG'
K12MG_sb = K12MG_sb.drop(['block_by_start','block_by_end', 'Single_block', 'block_order','diff','overlaps'], axis=1)

K12DH_sb['strain'] = 'K12DH'
K12DH_sb = K12DH_sb.drop(['block_by_start','block_by_end', 'Single_block', 'block_order','diff','overlaps'], axis=1)

ATCC_final['strain'] = 'ATCC'
ATCC_final = ATCC_final.drop(['block_by_start','block_by_end', 'Single_block', 'block_order','diff','overlaps'], axis=1)

BW25113_final['strain'] = 'BW25113'
BW25113_final = BW25113_final.drop(['block_by_start','block_by_end', 'Single_block', 'block_order','diff','overlaps'], axis=1)

df = pd.concat([K12DH_sb,K12MG_sb,BW25113_final, ATCC_final])

len(df) == len(ATCC_final) + len(BW25113_final) + len(K12DH_sb) + len(K12MG_sb)





True

In [162]:
df.to_csv('Sample_final_df_1025.csv',index=False)

In [163]:
len(df)

12248

In [164]:
len(BW25113_sb)

3040

In [165]:
len(ATCC_final)

3181

In [166]:
len(K12DH_sb)

3058

In [167]:
len(K12MG_sb)

2969

In [168]:
len(BW25113)

4077